In [1]:
# import libraries
import os
import pandas as pd

In [2]:
# must match a reference ontology
# path in final version will be /opt/data/dim/
concept = pd.read_csv('volumes/data/dim/CONCEPT.csv.gz', header=0, sep='\t', low_memory=False)

In [3]:
concept = concept[['concept_code', 'concept_id']]

In [4]:
concept.drop_duplicates(inplace=True)

In [5]:
concept.rename(columns= {'concept_code': 'cpt'}, inplace=True)

In [6]:
# make a control file to iterate through
# path in final version will be /opt/data/raw/
path = 'volumes/data/raw/'
files = os.listdir(path)
csv_files = [x for x in files if x.endswith(".csv.gz")]

# removes '.csv.gz' (7 characters)
hospital_ids = [int(x[:-7]) for x in csv_files] 
control = pd.DataFrame({'file': csv_files, 'hospital_id': hospital_ids})

In [7]:
#cross reference the control file with the static dimension table
# path in final version will be /opt/dta/dim/
dim = pd.read_csv('volumes/data/dim/hospital.csv', usecols=['hospital_id', 'affiliation'])
control = control.merge(dim, how= 'left', on='hospital_id')
control.sort_values(by='hospital_id', inplace=True, ignore_index=True)

In [8]:
control.head(10)

,file,hospital_id,affiliation
0,1.csv.gz,1,Independent
1,2.csv.gz,2,Independent
2,3.csv.gz,3,Independent
3,7.csv.gz,7,Independent
4,8.csv.gz,8,Independent
5,10.csv.gz,10,Independent
6,12.csv.gz,12,Appalachian Regional Healthcare System
7,13.csv.gz,13,Appalachian Regional Healthcare System


In [ ]:
## loop (PENDING)

In [14]:
# read in the data
df = pd.read_csv('volumes/data/raw/' + control.file[0], low_memory=False)
display(df.head())

,Plan,Procedure Code,Price Tier,Procedure Description,Unit Price,Revenue Code,CPT/HCPCS Code,CPT/HCPCS Desc,NDC Code,Rx Unit Multiplier,Cash Discount Price,Minimum Amount,Maximum Amount,Inpatient Amount,Outpatient Amount,Ambulatory Surgical Amount,Emergency Amount,Professional Fee Amount,Observation Amount,Urgent Care Amount
0,ACS CONSULT SVC INC,1325131,NaN,60505-0847-03 - fluticasone nasal 0.05 mg/inh Spr,NaN,637.0,J3490,Drugs unclassified injection,60505-0847-03,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ACS CONSULT SVC INC,1325133,NaN,00009-0047-26 - methylPREDNISolone 125 mg Pow,50,636.0,J2930,Methylprednisolone injection,00009-0047-26,1,30,28.15,46.5,46.5,46.5,46.5,46.5,NaN,46.5,NaN
2,ACS CONSULT SVC INC,1325134,NaN,00009-0039-32 - methylPREDNISolone 40 mg Pow,50,636.0,J2920,Methylprednisolone injection,00009-0039-32,1,30,28.15,46.5,46.5,46.5,46.5,46.5,NaN,46.5,NaN
3,ACS CONSULT SVC INC,1325135,NaN,17478-0209-10 - ketorolac ophthalmic 0.5% Sol,NaN,637.0,J3490,Drugs unclassified injection,17478-0209-10,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ACS CONSULT SVC INC,1325137,NaN,63323-0658-20 - chlorothiazide 0.5 g Pow,"1,143.45",636.0,J1205,Chlorothiazide sodium inj,63323-0658-20,1,686.07,643.76,"1,063.41","1,063.41","1,063.41","1,063.41","1,063.41",NaN,"1,063.41",NaN


In [21]:
# hospital specific configuration
out = pd.DataFrame({
    'cpt': df['CPT/HCPCS Code'],
    'gross': df['Unit Price'],
    'cash': df['Cash Discount Price'],
    'max': df['Maximum Amount'],
    'min': df['Minimum Amount']
})
out.drop_duplicates(inplace=True)

(8805, 5)